In [0]:
from google.colab import drive
drive.mount("/gdrive",force_remount=True)

Mounted at /gdrive


In [0]:
%cd "/gdrive/My Drive/Compiler Design/CD-py"

/gdrive/My Drive/Compiler Design/CD-py


In [0]:
from left_recursion import removeLeftRecursion,printGrammar
from first_follow import first,follow

In [0]:
def add_production(key,production,table):
    if key in table.keys():
        print("Error: Grammar is not LL1")
        exit()
    else:
        table[key] = production


In [0]:
def first_of_rhs(p,G):
    first = set()
    for char in p:
        if char in G["terminals"]:
            first.add(char)
            break
        elif char in G["symbols"]:
            f_char = first(char,G)
            first = first.union(f_char)
            if 'E' not in f_char:
                break
            else:
                if char != p[-1]:
                    first-=set('E') 
        elif char == 'E':#epsilon
            if char == p[-1]:
                first.add('E')
    return first
    

In [0]:
def generateParserTable(G):
    table = {} # key is a tuple - (symbol,terminal) value = production(list of string)
    for symbol,production in G["productions"].items():
        for p in production:
            first = first_of_rhs(p,G)
            for char in first:
                if char == "E": #epsilon
                    fol = follow(symbol,G)
                    for f in fol:
                        add_production((symbol,f),p,table)
                else:
                    add_production((symbol,char),p,table)
    return table

In [0]:
def error(string):
    print(f"{string} cannot be generated from given grammar")
    #exit()
    
def LL1parser(G,string):
    G = leftFactoring(G) #removing left recurssion if any
    table = generateParserTable(G)    
    string = string + '$'
    stack = ['$','S']
    cur = 0
    while(cur<len(string)):
        top = stack[-1]
        if top in G["symbols"]:
            if (top,string[cur]) in table.keys():
                prod = table[(top,string[cur])]
                if prod[0] == 'E':
                    stack = stack[:-1]
                else:
                    stack = stack[:-1] + prod[-1::-1] #popping and pushing prod on stack
            else:
                error(string)
                break
        elif top in G["terminals"]:
            if top == string[cur]:
                stack = stack[:-1] #pop 
                cur+=1
            else:
                error(string)
                break                
        else:
            if top == string[cur] == '$':
                print("string accepted")
                break
            else:
                error(string)
                break
            

In [0]:
G = {
    "start":'S',
    "terminals":set(['(',')']),
    "symbols":set(["S"]),
    "productions":{
        "S":[['(','S',')'],['E']]
    }
}


In [0]:
LL1parser(G,"(((()))))")

(((()))))$ cannot be generated from given grammar


In [0]:
LL1parser(G,"(((())))")

string accepted
